## [Copernicus GFM API](https://api.gfm.eodc.eu/v1/)
* [Wiki](https://extwiki.eodc.eu/GFM/PUM/DataAccess/REST-APIs)
* [REST API Tutorial](https://realpython.com/api-integration-in-python/)

In [41]:
from os.path import join
from event import Event
from ast import literal_eval
import requests
import json
import wget
import pandas as pd
import os
import zipfile
from io import BytesIO
from glob import glob

# create an access token by logging-in
headers =  {"Content-Type":"application/json"}
token = {
    "email": "alison.peard@gmail.com",
    "password": "Luckyd0ggygfm"
}

api_url = 'https://api.gfm.eodc.eu/v1/'
response = requests.post(f"{api_url}/auth/login", data=json.dumps(token), headers=headers)
access_token = response.json()['access_token']
token_type = response.json()['token_type']
user_id = response.json()['client_id']
print(f"Access token: {token_type} {access_token}")
print(f"User ID: {user_id}")

Access token: Bearer jvMQXKv1QGL8gtFc8bfXZFOPqUN0YdpJRLyVeNPO2f
User ID: 6SEgIQ6yF6LXa2zEAlCjxcbw


In [42]:
def create_aoi(storm, region, subregion, geoJSON, api_url, user_id, token_type, access_token):
    
    # create aoi based-on event data
    headers = {
        'accept': 'application/json',
        'Authorization': f"{token_type} {access_token}",
        'Content-Type': 'application/json'
    }

    body = {
        "aoi_name": f"{storm}_{region}_{subregion}",
        "description": f"Storm {storm.capitalize()} in {region.capitalize()}, subregion {subregion}",
        "user_id": user_id,
        "geoJSON": geoJSON

    }

    response = requests.post(f"{api_url}/aoi/create", headers=headers, data=json.dumps(body))
    aoi_id = response.json()['aoi_id']
    
    return aoi_id


# retrive the new aoi
def retrieve_aoi(aoi_id, api_url, token_type, access_token):
    headers = {
        'accept': 'application/json',
        'Authorization': f"{token_type} {access_token}",
        'Content-Type': 'application/json'
    }

    response = requests.get(f"{api_url}/aoi/{aoi_id}", headers=headers)
    response.json()
    

def get_product(aoi_id, startdate, enddate, api_url, token_type, access_token):# retrive the relevant product
    headers = {
        'accept': 'application/json',
        'Authorization': f"{token_type} {access_token}",
        'Content-Type': 'application/json'
    }

    query = {
        'time': 'range',
        'from': f"{startdate}T00:00:00",
        'to': f"{enddate}T00:00:00"
    }

    response = requests.get(f"{api_url}/aoi/{aoi_id}/products", params=query, headers=headers)
    # get first product, TODO: iterate through if more
    print(f"Found {len(response.json()['products'])} products.")
    product_ids = [product['product_id'] for product in response.json()['products']]
    product_times = {product['product_id']: product['product_time'] for product in response.json()['products']}
    
    
    return response.json(), product_ids, product_times

# get the product that seems to have the most exposed land using statistics
def get_most_affected_by_pop(product_ids, product_times, api_url, token_type, access_token):
    affected_landcovers = {}
    affected_populations = {}
    for i, product_id in enumerate(product_ids):
        headers = {
            'accept': 'application/json',
            'Authorization': f"{token_type} {access_token}",
            'Content-Type': 'application/json'
        }

        response = requests.get(f"{api_url}/reporting/report_statistics/{product_id}", headers=headers)
        affected_landcovers[product_id] = response.json()['affected_landcover']
        affected_populations[product_id] = response.json()['affected_population']

    affected_landcovers = {key: sum([*value.values()]) for key, value in affected_landcovers.items()}
    product_id = max(affected_populations, key=affected_populations.get)
    product_time = product_times[product_id]
    
    return product_id, product_time, max(affected_populations.values())


# TIDY
def check_for_notifications(storm, region, subregion, startdate, enddate, geoJSON, api_url, user_id, token_type, access_token):
    
    aoi_id = create_aoi(storm, region, subregion, geoJSON, api_url, user_id, token_type, access_token)
    response, _, _ = get_product(aoi_id, startdate, enddate, api_url, token_type, access_token)
    notifications = {product['product_id']: (product['notification_seen'], product['product_time'])  for product in response['products'] if product['notification_seen'] is not None}
    
    return notifications

# TIDY
def check_most_affected_pop(storm, region, subregion, startdate, enddate, geoJSON, api_url, user_id, token_type, access_token):
    
    aoi_id = create_aoi(storm, region, subregion, geoJSON, api_url, user_id, token_type, access_token)
    _, product_ids, product_times = get_product(aoi_id, startdate, enddate, api_url, token_type, access_token)
    product_id, product_time, affected_pop = get_most_affected_by_pop(product_ids, product_times, api_url, token_type, access_token)
    
    return product_id, product_time, affected_pop


def download_product(product_id, storm, region, subregion, api_url, token_type, access_token, wd):

    headers = {
        'accept': 'application/json',
        'Authorization': f"{token_type} {access_token}",
        'Content-Type': 'application/json'
    }

    response = requests.get(f"{api_url}/download/product/{product_id}", headers=headers)
    download_link = response.json()['download_link']

    # download using wget

    local_filename = f"{storm}_{region}_{subregion}_{product_id}.zip"
    wget.download(download_link, out=join(wd, 'copernicus_gfm', local_filename))
    
    print(f"\nSaved as {local_filename}")
    return local_filename

def tidy_download(local_filename, product_time, storm, region, subregion, wd, unzip=True):
    
    if unzip:
        # extract & tidy up directory
        timestamp = product_time.replace(':', '')
        with zipfile.ZipFile(join(wd, 'copernicus_gfm', local_filename)) as zipped:
            zipped.extractall(path=join(wd, 'copernicus_gfm', local_filename.replace('.zip', '')))
            os.remove(join(wd, 'copernicus_gfm', local_filename))

    # only keep floodfile, and rename
    for file in glob(join(wd, 'copernicus_gfm', f"{storm}_{region}_{subregion}/*")):
        if 'FLOOD' in file:
            path = join(*file.split('/')[:-1])
            extension = file.split('.')[-1]
            os.rename(file, join(path, f"{storm}_{region}_{subregion}_FLOOD{timestamp}.{extension}"))
        elif 'OBSWATER' in file:
            path = join(*file.split('/')[:-1])
            extension = file.split('.')[-1]
            os.rename(file, join(path, f"{storm}_{region}_{subregion}_OBSWATER{timestamp}.{extension}"))
        elif 'OBSWATER' in file:
            path = join(*file.split('/')[:-1])
            extension = file.split('.')[-1]
            os.rename(file, join(path, f"{storm}_{region}_{subregion}_OBSWATER{timestamp}.{extension}"))
        else:
            os.remove(file)

In [43]:
bd = '.'
wd = join(bd, "..", "data")

current_datasets = pd.read_csv(join(wd, 'csvs', 'current_datasets.csv'))
current_datasets = current_datasets[current_datasets.event=='batsirai']

# Single product

In [44]:
bd = '.'
wd = join(bd, "..", "data")
storm = "rai"
region = "hinunangan"
nsubregions = 1
subregion = 0

# # choose manually
# product_id = '52211056158fee25d5cd3d26e62ab405dab500e02d45733ca9488f8275249b0e'
# product_time = '2022-03-12T02651:32'

# load event GeoDataFrame (as in loop)
event = Event(storm, region, nsubregions, wd, bd)
event.make_grids()
event.get_all_features(subregion)

startdate = event.startdate
enddate = event.enddate
geoJSON = literal_eval(event.aoi_lonlat[subregion]['geometry'].to_json())
geoJSON = geoJSON['features'][0]['geometry']

In [45]:
aoi_id = create_aoi(storm, region, subregion, geoJSON, api_url, user_id, token_type, access_token)
retrieve_aoi(aoi_id, api_url, token_type, access_token)
response, product_ids, product_times = get_product(aoi_id, startdate, enddate, api_url, token_type, access_token)
response['products']

Found 6 products.


[{'product_id': '1ce3cf75ebce65623fe2ace6061808761ed5a5bb841d2a75b29e987bad961bf2',
  'product_time': '2021-12-13T21:30:52',
  'notification_seen': None,
  'latest_product': False,
  'layer_boundary': {'type': 'Polygon',
   'coordinates': [[[125.371063, 10.032187],
     [125.680931, 11.544516],
     [123.4207, 11.986881],
     [123.123077, 10.478406],
     [125.371063, 10.032187]]]}},
 {'product_id': '205282c6de3b467bb178d95059ac39fb316ad56a97c5d7834d9ee1ccd678e4bf',
  'product_time': '2021-12-20T21:22:53',
  'notification_seen': None,
  'latest_product': False,
  'layer_boundary': {'type': 'Polygon',
   'coordinates': [[[127.242813, 9.127525],
     [127.553329, 10.639739],
     [125.295479, 11.085361],
     [124.996269, 9.577056],
     [127.242813, 9.127525]]]}},
 {'product_id': '60ff11ee3b5892e96a82bf18c382eb08fd908aa9e7345fc34a3936ad9fd43be1',
  'product_time': '2021-12-13T21:30:52',
  'notification_seen': None,
  'latest_product': False,
  'layer_boundary': {'type': 'Polygon',
   '

In [46]:
options = {i: {'product_id': product['product_id'],
               'product_time': product['product_time']} for i, product in enumerate(response['products'])}

options

{0: {'product_id': '1ce3cf75ebce65623fe2ace6061808761ed5a5bb841d2a75b29e987bad961bf2',
  'product_time': '2021-12-13T21:30:52'},
 1: {'product_id': '205282c6de3b467bb178d95059ac39fb316ad56a97c5d7834d9ee1ccd678e4bf',
  'product_time': '2021-12-20T21:22:53'},
 2: {'product_id': '60ff11ee3b5892e96a82bf18c382eb08fd908aa9e7345fc34a3936ad9fd43be1',
  'product_time': '2021-12-13T21:30:52'},
 3: {'product_id': '73c0eefbb3923af446f02b977b4671bce13fc53bfb74599ba63a336fc50a4bd5',
  'product_time': '2021-12-14T21:23:44'},
 4: {'product_id': '92ecb98514d630bc7faf0ae3002aa83a0f910bab0a14c589c2f25d46c58f69ec',
  'product_time': '2021-12-14T21:23:44'},
 5: {'product_id': 'ad573dba725b04d418e1658b660ae2e3e705dac337e6bad8eb9a8d1063268b78',
  'product_time': '2021-12-20T21:22:53'}}

In [47]:
option = options[5]  # to just download one
# for _, option in options.items():
local_filename = download_product(option['product_id'], storm, region, subregion, api_url, token_type, access_token, wd)
tidy_download(local_filename, option['product_time'], storm, region, subregion, wd, unzip=True)

100% [......................................................] 6407298 / 6407298
Saved as rai_hinunangan_0_ad573dba725b04d418e1658b660ae2e3e705dac337e6bad8eb9a8d1063268b78.zip


# Look which product has greatest extents

In [ ]:
# check if any of my sets produced a GFM notification
best_products = {}
for _, row in current_datasets.iterrows():
    # define event
    storm = row['event']
    region = row['region']
    nsubregions = int(row['nsubregions'])
    
    notifications = {}
    affected_pops = {}
    for subregion in range(nsubregions):
        event_str = f"{storm}_{region}_{subregion}"
        # load event GeoDataFrame (as in loop)
        event = Event(storm, region, nsubregions, wd, bd)
        event.make_grids()
        event.get_all_features(subregion)

        startdate = event.startdate
        enddate = event.enddate
        geoJSON = literal_eval(event.aoi_lonlat[subregion]['geometry'].to_json())
        geoJSON = geoJSON['features'][0]['geometry']
        
#         notifications[event_str] = check_for_notifications(storm, region, subregion,
#                                                            startdate, enddate, geoJSON, api_url,
#                                                            user_id, token_type, access_token)

        try:
            affected_pops[event_str] = check_most_affected_pop(storm, region, subregion, startdate, enddate,
                                                               geoJSON, api_url, user_id, token_type, access_token)

        except Exception as e:
            print(e)
            
        print(f"{event_str}: {affected_pops[event_str]}")
        best_products[event_str] = affected_pops[event_str]

In [ ]:
bd = '.'
wd = join(bd, "..", "data")

for event, product in best_products.items():
    storm = event.split('_')[0]
    region = event.split('_')[1]
    nsubregions = 6
    subregion = event.split('_')[2]
    
    if subregion == '0':
        product_id = product[0]
        product_time = product[1]

        local_filename = download_product(product_id, storm, region, subregion, api_url, token_type, access_token, wd)
        tidy_download(local_filename, storm, region, subregion, wd, unzip=True)

        print(f"Downloaded {event} as {local_filename}.")